In [8]:
import numpy as np
import wave
import os
import librosa

def IAV(data,frame_size):
    value=0.0;

    for i in range (frame_size):
        value += abs(data[i])
    
    return value

def getIAVThreshold(y,length,frame_size):
    IAV_data_size= 0
    Max_length = 0
    Min_length = 0
    threshold = 0.0
    Max_avg = 0.0
    Min_avg = 0.0
    
    IAV_data_size = int(length / frame_size)
    Max_length = int(IAV_data_size * 0.2)
    Min_length = int(IAV_data_size * 0.2)
    
    IAV_data = np.zeros(IAV_data_size, dtype=np.float32)
    Max = np.zeros(Max_length, dtype=np.float32)
    Min = np.zeros(Min_length, dtype=np.float32)
    
    j = 0
    
    for i in range(0,length,frame_size):
        if (j >= IAV_data_size):
            break
            
        data_sub = y[i:i+frame_size]
        IAV_data[j] = IAV(data_sub,frame_size)
        j = j + 1
        
    sort_indices = np.argsort(IAV_data)[::-1]
    IAV_data[:] = IAV_data[sort_indices]
        
    for i in range(Max_length):
        Max[i] = IAV_data[i]
        
    IAV_data.sort()
    
    for i in range(Min_length):
        Min[i] = IAV_data[i]
                          
    threshold = 0.0
    
    Max_avg = np.mean(Max)
    Min_avg = np.mean(Min)
    
    if(Max_avg*0.7 > Min_avg):
        threshold = (Max_avg - Min_avg) * 0.1
        threshold = threshold + Min_avg        
    else:
        threshold = Max_avg*0.2
        
    return threshold

def getStartPointEndPoint(y,sr,length,Window_size):
    start_point = 0
    end_point = 0
    
    IAV_th = getIAVThreshold(y,length,Window_size)
    th = IAV_th/Window_size*2
    
    # start point
    for i in range(0,length-1,int(Window_size/3)):
        iav = IAV(y[i:i+Window_size],Window_size)
        
        if iav > IAV_th:
            start_point = i 
            for j in range(Window_size):
                if (abs(y[i+j]) > th):
                    start_point = i+j
                    break
            break
    
    
    if (start_point - Window_size > 0) == 1:
        start_point = start_point - Window_size
    
    # end point
    for i in range(length-1,0,-int(Window_size/3)):
        iav = IAV(y[i-Window_size:i],Window_size)
        
        if iav > IAV_th:
            end_point = i
            for j in range(Window_size):
                if (abs(y[i-j]) > th):
                    end_point = i-j
                    break
            break
    
    
    if (end_point + Window_size < length) == 1:
        end_point = end_point + Window_size
    
    return start_point, end_point

def load_wav(path, sr):
    return librosa.core.load(path, sr=sr)[0]

In [5]:
for file in os.listdir(wav_dir):
    file_path = os.path.join(wav_dir, file)
    print(file_path)
print(len(os.listdir(wav_dir)))

./before3/m3 (26).wav
./before3/m3 (6).wav
./before3/m3 (8).wav
./before3/m3 (3).wav
./before3/m3 (15).wav
./before3/m3 (25).wav
./before3/m3 (12).wav
./before3/m3 (9).wav
./before3/m3 (30).wav
./before3/m3 (1).wav
./before3/m3 (24).wav
./before3/m3 (2).wav
./before3/m3 (10).wav
./before3/m3 (27).wav
./before3/m3 (4).wav
./before3/m3 (18).wav
./before3/m3 (14).wav
./before3/m3 (16).wav
./before3/m3 (19).wav
./before3/m3 (29).wav
./before3/m3 (5).wav
./before3/m3 (21).wav
./before3/m3 (28).wav
./before3/m3 (11).wav
./before3/m3 (7).wav
./before3/m3 (22).wav
./before3/m3 (17).wav
./before3/m3 (20).wav
./before3/m3 (23).wav
./before3/m3 (13).wav
30


In [10]:
wav_dir='./before3'
sr=16000
file_path = os.path.join(wav_dir, file)
wav = load_wav(file_path, sr=sr)
print(wav)
print(len(wav))

[-0.0045166  -0.00595093 -0.00546265 ...  0.00396729  0.00476074
  0.00546265]
103606


In [11]:
# 여러개파일 변환할 때

wav_dir='./before4'
sr=16000

num_file = len(os.listdir(wav_dir))

for i in range(num_file) :
    for file in os.listdir(wav_dir):
        file_path = os.path.join(wav_dir, file)
        wav = load_wav(file_path, sr=sr)
        Window_size = 500
        length = len(wav)
        start_point, end_point = getStartPointEndPoint(wav,sr,length,Window_size)
        y = wav[start_point:end_point]
        librosa.output.write_wav(os.path.join('./after', file), y, sr)

In [104]:
#한개씩 변환할 때

one_file_path = '.wav' #파일이름

wav = load_wav(one_file_path, sr=sr)
Window_size = 500
length = len(wav)
start_point, end_point = getStartPointEndPoint(wav,sr,length,Window_size)
y = wav[start_point:end_point]
librosa.output.write_wav('./after', y, sr)

In [19]:
import matplotlib.pyplot as plt

y1 = wav

plt.plot(y1)
plt.show()

<Figure size 640x480 with 1 Axes>